In [1]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
from math import log, e
import pydotplus

In [2]:
def retrieve_all(Ts): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  =motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO MOTIFS
   # print('Motifs starting position: '+str(mot)+ ' Motifs values (min distances): '+str(motif_dist))
   # print(" ")

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [3]:
#riceve la lista di coppie dei motifs per ogni record(Ts), e resittuisce lista di valori singoli

def motifsFilter(MotifsList): 
    l2=np.array([])
    for i in range (len(MotifsList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(MotifsList['Motif'].loc[i])
        #print(numMotif)
        for j in range (numMotif): # per ogni lista di motif
            l1=MotifsList['Motif'].loc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista 
    
        MotifsList['Motif'].loc[i]=l2
        l2=np.array([]) #svuoto array
    
    return MotifsList

In [18]:
#carico dataset in dataFrame
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
df = pd.DataFrame(dataset[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
#trasformo da stringa a numero il campo target 
le = LabelEncoder()
num_classes = le.fit_transform(df['target'])
df['target']=num_classes
df['TsIndex']=np.arange(len(df))

print(df)

        att1      att2      att3      att4      att5      att6      att7  \
0  -0.464276 -0.555048 -0.842843 -0.865895 -0.936396 -0.817270 -0.263612   
1  -0.896972 -0.685686 -1.351382 -1.458667 -1.165346 -1.403929 -1.821800   
2  -0.464696 -0.567739 -0.032023 -0.635046 -0.602826 -0.266856 -0.267061   
3  -0.187190 -0.620808 -0.815661 -0.521398 -0.790423 -0.967517 -1.487006   
4  -1.136017 -1.319195 -1.844624 -0.788692 -0.251715 -1.487603 -0.668764   
5  -0.568709 -0.888118 -0.047977 -0.038036 -0.483068 -1.094689 -0.953453   
6  -0.965839 -1.644583 -1.805935 -1.499658 -0.920088 -1.115706 -0.650195   
7  -0.264302 -0.290925 -0.598567 -0.518863 -0.488594 -0.407091 -1.071489   
8  -0.803716 -0.048161 -1.299278 -1.072280 -1.004018 -0.511452 -1.173215   
9  -1.186422 -0.893197 -0.977737 -0.634109 -1.384820 -1.271926 -1.212171   
10 -0.653308 -1.295185 -0.457384 -1.092036 -1.682346 -0.461811 -0.099362   
11 -0.541876 -1.016107 -1.079902 -1.185665 -0.792452 -0.754593 -0.057041   
12 -1.945327

In [19]:
window_size=5
#diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
diz={'Motif':[]}

#CALCOLO MOTIF E DISCORD E LI INSERISCO NEL DIZIONARIO
for i in range(30):
    Ts = np.array(df.iloc[i].values)
    mp,mot,motif_dist,dis = retrieve_all(Ts[0:128])
    diz['Motif'].insert(i, mot)

#GENERO DFRAME DA DIZIONARIO

MotifsList = pd.DataFrame(diz)
MotifsList=motifsFilter(MotifsList)

print('Motif estratti')
#print(MotifsList['Motif'])
print(MotifsList)

Motif estratti
            Motif
0     [9.0, 26.0]
1   [105.0, 54.0]
2    [17.0, 40.0]
3    [48.0, 12.0]
4          [27.0]
5    [45.0, 13.0]
6     [7.0, 37.0]
7     [2.0, 24.0]
8     [1.0, 29.0]
9     [50.0, 0.0]
10   [11.0, 17.0]
11    [31.0, 7.0]
12         [53.0]
13    [79.0, 6.0]
14         [59.0]
15    [2.0, 45.0]
16    [28.0, 3.0]
17   [14.0, 35.0]
18    [13.0, 3.0]
19    [19.0, 6.0]
20   [19.0, 39.0]
21         [26.0]
22   [18.0, 22.0]
23   [27.0, 42.0]
24    [8.0, 28.0]
25         [16.0]
26   [67.0, 59.0]
27   [16.0, 39.0]
28   [11.0, 41.0]
29    [6.0, 15.0]


In [6]:
def computeSubSeqDistance(dataset,motifDiscordList):
    
    #per ogni Ts calcolo Dprofile con ogni candidato e inserisco la distanza minima con candidato i-esimo nella colonna i-esima
    numberOfCandidates=0
    for i in range(len(motifDiscordList)):
            numberOfCandidates+=len(motifDiscordList['Motif'].loc[i])
    columnsList=np.arange(numberOfCandidates)
    columnsList2=['TsIndex','class']
    for i in columnsList:
        columnsList2.append(str(i)) 
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(dataset)))

    
    for i in range(len(dataset)):
        TsToCompare = np.array(dataset.iloc[i].values) #scandisco ogni Ts
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].loc[i]=i
        dfForDTree['class'].loc[i]=classValue
        counter=0
        for j in range(len(motifDiscordList)):
            numMotif=len(motifDiscordList['Motif'].loc[j])
            for k in range(numMotif):
                l1=motifDiscordList['Motif'].loc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[str(counter)].loc[i]=minValueFromDProfile
                counter+=1
    return dfForDTree,numberOfCandidates,columnsList2
                
                

In [44]:
def split(dataset,columnsList,attribute,value): 
    dizLeft=pd.DataFrame(columns=columnsList)
    dizRight=pd.DataFrame(columns=columnsList)
    for i in range(len(dataset)):
        if dataset.iloc[i][attribute] < value:
            dizLeft = dizLeft.append(dataset.iloc[i], ignore_index=True)
        else:
            dizRight = dizRight.append(dataset.iloc[i], ignore_index=True)
    return dizLeft, dizRight

In [55]:
#dataset (dframe): nella riga i: indice della ts di appartenenza, distanza tra candidato e Ts, e classe di appartenenza di Ts
#calcola entropia di un dataset basandosi sul num di classi esistenti
def computeEntropy(dataset, num_classes_unique):
    numPattern=len(dataset)
    print('numPattern: '+str(numPattern))
    relativeFrequencies=list()
    counter=0 #num pattern di ogni classe nel dataset
    #ottengo frequenze relative
    for i in (num_classes_unique):
        for j in range(numPattern):
            if(dataset.iloc[j]['class']==i):
                counter+=1
        relativeFrequencies.append(counter/numPattern)
        counter=0
    #calcolo entropia dataset
    print('relativeFrequencies: '+str(relativeFrequencies))
    entropy=0
    for k in (relativeFrequencies):
        if(k!=0):
            entropy+=(k*log(k,2))
    return -entropy

In [56]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli
def computeGain(DatasetParent,Dleft,Dright,num_classes_unique):
    entropyParent=computeEntropy(DatasetParent,num_classes_unique)
    entropyLeft=computeEntropy(Dleft,num_classes_unique)
    entropyRight=computeEntropy(Dright,num_classes_unique)
    gain=entropyParent
    summation=( ((len(Dleft)/len(DatasetParent))*entropyLeft) +  ((len(Dright)/len(DatasetParent))*entropyRight) )
    print('entropyParent: '+str(entropyParent))
    print('SUMMATION: '+str(summation))
    gain=gain-summation
    return gain

In [57]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli
def computeGain2(entropyParent,lenDatasetParent,Dleft,Dright,num_classes_unique):
    entropyLeft=computeEntropy(Dleft,num_classes_unique)
    entropyRight=computeEntropy(Dright,num_classes_unique)
    gain=entropyParent
    summation=( ((len(Dleft)/lenDatasetParent)*entropyLeft) +  ((len(Dright)/lenDatasetParent)*entropyRight) )
    print('entropyParent: '+str(entropyParent))
    print('SUMMATION: '+str(summation))
    gain=gain-summation
    return gain

In [65]:
def findBestAttributeValue(dataset,entropyParent):
    #cerca e restituisce attributo migliore su cui splittaree relativo valore ottimale (optimal split point)
    bestGain=0
    actualGain=0
    bestvalueForSplit=0
    y = dfForDTree['class'].values
    y=y.astype('int')
    
    #trovo best Attribute
    vecMutualInfo=mutual_info_classif(dataset.iloc[:, 2::], y, discrete_features=False)
    bestValueMutualInfo=vecMutualInfo.max()
    indexBestAttribute=np.argwhere(vecMutualInfo==bestValueMutualInfo)+2 #indice dell'attributo con best gain 
    
    #splitto su tale attributo e trovo il valore ottimale
    num_classes_unique=np.unique(num_classes) 
    for i in range (len(dataset)):
        testValue=dataset.iloc[i][indexBestAttribute]
        if(testValue>0):
            Dleft,Dright=split(dataset,columnsList,indexBestAttribute,testValue)
            actualGain=computeGain2(entropyParent,len(dataset),Dleft,Dright,num_classes_unique)
            if(actualGain > bestGain):
                bestGain=actualGain
                bestvalueForSplit=testValue

    return indexBestAttribute,bestvalueForSplit

In [69]:
#calcolo distanze per ogni Ts
dfForDTree,numberOfCandidates,columnsList=computeSubSeqDistance(df,MotifsList)

num_classes_unique=np.unique(num_classes)
entropyParent=computeEntropy(dfForDTree, num_classes_unique)
Indice,valore=findBestAttributeValue(dfForDTree,entropyParent)
print('INDICE: '+str(Indice))
print('VALORE: '+str(valore))

numPattern: 30
relativeFrequencies: [0.3333333333333333, 0.4, 0.26666666666666666]
numPattern: 2
relativeFrequencies: [0.0, 0.5, 0.5]
numPattern: 28
relativeFrequencies: [0.35714285714285715, 0.39285714285714285, 0.25]
entropyParent: 1.5655962303576019
SUMMATION: 1.5227141536441933
numPattern: 14
relativeFrequencies: [0.42857142857142855, 0.5, 0.07142857142857142]
numPattern: 16
relativeFrequencies: [0.25, 0.3125, 0.4375]
entropyParent: 1.5655962303576019
SUMMATION: 1.4293528173745418
numPattern: 12
relativeFrequencies: [0.3333333333333333, 0.5833333333333334, 0.08333333333333333]
numPattern: 18
relativeFrequencies: [0.3333333333333333, 0.2777777777777778, 0.3888888888888889]
entropyParent: 1.5655962303576019
SUMMATION: 1.4551938549015095
numPattern: 20
relativeFrequencies: [0.4, 0.4, 0.2]
numPattern: 10
relativeFrequencies: [0.2, 0.4, 0.4]
entropyParent: 1.5655962303576019
SUMMATION: 1.5219280948873621
numPattern: 26
relativeFrequencies: [0.38461538461538464, 0.34615384615384615, 0.26